In [1]:
#import all required packages..

import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.stats import pearsonr
from sklearn.metrics import pairwise_distances

In [2]:
#import Rating, Movie and Tags Data basically Movie and Tag are just mapping for Movie and Tag
# we are using only Rating Frame as we need to deal with movie and rating with user...

Ratings=pd.read_csv("ratings.csv",encoding="ISO-8859-1")
Movies=pd.read_csv("movies.csv",encoding="ISO-8859-1")
Tags=pd.read_csv("tags.csv",encoding="ISO-8859-1")

In [3]:
Ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [4]:
#Get movie rating count so it will be easy at end to rank movie when 2 movie get same rating....
#this is optional if you want can do it else skip this step.

#1. take groupby with respect to movie and get mean avg of each movie. 
movie_rating_count = Ratings.groupby('movieId')['rating'].count()

#2. convert into dataframe for better operation work.
movie_rating_count = pd.DataFrame(movie_rating_count)

#3. change column name.
movie_rating_count.columns = ['rating_count']

#4. create new column movieIUd
movie_rating_count['movieId'] = movie_rating_count.index
movie_rating_count.reset_index(drop=True)

#merging to Rating dataframe so we have collectively all information together..
Ratings = Ratings.merge(movie_rating_count,right_on='movieId',left_on='movieId')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: 'movieId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


In [5]:
Ratings.head()


,userId,movieId,rating,timestamp,rating_count
0,12882,1,4.0,1147195252,496
1,121987,1,4.5,1308152264,496
2,18127,1,4.0,1130946329,496
3,17318,1,4.0,1168807334,496
4,69519,1,4.0,1303980621,496


In [6]:
Movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
Tags.head()

,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
1,4085,12882,Eddie Murphy,1147195966
2,33660,12882,boxing,1147195514
3,1197,320,must show,1145964801
4,1396,320,must show,1145964810


In [8]:
#create a pivot matrix for user and movie based on rating value..
RatingMat = Ratings.pivot_table(index=['userId'],columns=['movieId'],values=['rating'],fill_value=0)

#keep a copy of original matrix for future refrence..
Original_RatingMat = RatingMat.copy()
RatingMat.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      9      10     11       
userId                                                                          
316        2.5    0.0    0.0    0.0    0.0    0.0    2.0    0.0    2.5    0.0   
320        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
359        5.0    0.0    0.0    0.0    0.0    5.0    0.0    0.0    4.0    4.0   
370        4.5    4.0    0.0    0.0    0.0    5.0    0.0    0.0    0.0    0.0   
910        5.0    4.0    3.5    0.0    3.5    3.5    0.0    0.0    0.0    4.0   

         ...                                                                   \
movieId  ...   106487 106489 106782 106920 109374 109487 111362 111759 112556   
userId   ...                                                                    
316      ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
320      ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
359      ...      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
370      ...      2.5    3.0    4.5    4.0    0.0    0.0    3.0    4.5    3.5   
910      ...      0.0    0.0    3.5    0.0    0.0    0.0    0.0    4.5    0.0   

                
movieId 112852  
userId          
316        0.0  
320        0.0  
359        0.0  
370        3.0  
910        0.0  

[5 rows x 2500 columns]

In [9]:
#mean normalize to adjust rating on same scale for all user i.e. mean 0 across rows.
# this is because for user1 best mean 5 but user2 best means 4.

RatingMat = RatingMat.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

#drop multilevel column for better usabiltity...
RatingMat.columns = RatingMat.columns.droplevel()
RatingMat.head(5)

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,0.46564,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,0.36564,-0.03436,0.46564,-0.03436,...,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436
320,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,...,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836
359,0.75440,-0.24560,-0.24560,-0.24560,-0.24560,0.75440,-0.24560,-0.24560,0.55440,0.55440,...,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560
370,0.77524,0.67524,-0.12476,-0.12476,-0.12476,0.87524,-0.12476,-0.12476,-0.12476,-0.12476,...,0.37524,0.47524,0.77524,0.67524,-0.12476,-0.12476,0.47524,0.77524,0.57524,0.47524
910,0.78888,0.58888,0.48888,-0.21112,0.48888,0.48888,-0.21112,-0.21112,-0.21112,0.58888,...,-0.21112,-0.21112,0.48888,-0.21112,-0.21112,-0.21112,-0.21112,0.68888,-0.21112,-0.21112


In [10]:

#user wise similarity with cosine...
user_similarity = cosine_similarity(RatingMat)
user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
user_sim_df.head(5)

#we can do it using pearson correlation as well there is no hard and fast rule.
# generally user-user approach we use pearson correlation and item-item we use cosine similarity


#user wise similarity with pearson coorelation...
# user_similarity = 1-pairwise_distances(RatingMat, metric="correlation")
# user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
# user_sim_df.head(5)


userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,
316,1.000000,0.299545,0.225689,0.215937,0.171846,0.229365,0.247069,0.210951,0.007233,0.206959,...,0.290918,0.240003,0.035299,0.215968,0.367558,0.045559,0.278043,0.193846,0.017233,0.149235
320,0.299545,1.000000,0.183217,0.187426,0.157678,0.242885,0.160790,0.184277,-0.014179,0.197965,...,0.229539,0.323584,0.123649,0.170613,0.273515,0.067225,0.195021,0.180317,0.037405,0.096617
359,0.225689,0.183217,1.000000,0.218920,0.188056,0.239563,0.210592,0.168052,0.058551,0.252456,...,0.244404,0.190439,0.010915,0.173494,0.209795,0.011065,0.205926,0.120846,0.053407,0.142030
370,0.215937,0.187426,0.218920,1.000000,0.180415,0.323008,0.142359,0.136277,0.095462,0.291159,...,0.279538,0.166676,0.102677,0.220298,0.228517,0.061691,0.272535,0.079117,0.050965,0.127138
910,0.171846,0.157678,0.188056,0.180415,1.000000,0.257296,0.209615,0.192566,0.040182,0.310700,...,0.205902,0.153877,0.007079,0.211484,0.177860,-0.017890,0.174783,0.052333,0.066402,0.147138


In [11]:
# 1. select any user randomly for example we have selected userid 359 
#Get a list of movie user already watched so we can remove from our recommendation list..

curr_user_rated_movie = Ratings[(Ratings.userId == 359) & (Ratings.rating != 0)]['movieId']
curr_user_rated_movie = pd.DataFrame(curr_user_rated_movie,columns=['movieId'])

In [12]:
# put similarity of current user i.e. 359 in a dataframe because later we need for weighted average..

curr_user_similarity = pd.DataFrame(user_sim_df.loc[359])

#just changing column name as similarity for better readability.
curr_user_similarity.rename(columns={359:'similarity'},inplace=True)
curr_user_similarity.head()

,similarity
userId,
316,0.225689
320,0.183217
359,1.000000
370,0.218920
910,0.188056


In [13]:
# Remove movieId which user already watched from our rating matrix..
# above we have calculated which movie user has watched.
# subtract or negeate from rating matrix we get still not watched movie..

RatingMatTranspose = RatingMat.T
RatingMatTranspose = RatingMatTranspose.loc[~RatingMatTranspose.index.isin(curr_user_rated_movie.movieId.tolist())]

In [14]:
RatingMatTranspose.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
movieId,,,,,,,,,,,,,,,,,,,,,
2,-0.03436,-0.01836,-0.2456,0.67524,0.58888,-0.076489,0.51984,0.54344,-0.006622,-0.15832,...,-0.13564,0.66612,-0.010178,0.66468,-0.03116,-0.00772,0.2706,0.05636,-0.0046,-0.11444
3,-0.03436,-0.01836,-0.2456,-0.12476,0.48888,-0.076489,-0.08016,-0.05656,-0.006622,-0.15832,...,-0.13564,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,0.05636,-0.0046,-0.11444
4,-0.03436,-0.01836,-0.2456,-0.12476,-0.21112,-0.076489,-0.08016,-0.05656,-0.006622,-0.15832,...,-0.13564,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,-0.14364,-0.0046,-0.11444
5,-0.03436,-0.01836,-0.2456,-0.12476,0.48888,-0.076489,0.61984,-0.05656,-0.006622,-0.15832,...,-0.13564,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,-0.14364,-0.0046,-0.11444
7,0.36564,-0.01836,-0.2456,-0.12476,-0.21112,-0.076489,-0.08016,-0.05656,-0.006622,-0.15832,...,0.56436,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,-0.14364,-0.0046,-0.11444


In [15]:
#when we have taken pivot table it created multi level column name , so drop for better understanding..

Original_RatingMat.columns = Original_RatingMat.columns.droplevel()

In [16]:
'''
calculate the weighted average a movie per user so later we can show a user k specific - 
avg specific predicition rating for a movie

formulas :- sum(w[1]*user1+w[2]*user2.....w[n]*userN) / w[1]+w[2]....w[n] 
'''           
Weighed_avg = []

for movieId in RatingMatTranspose.index:
    
    '''
     User not watched movie consider as 0
     we need to remove those weight where user has not watched movie in weighted sum.
     In weighted sum in denominator we need to sum all weights
     we will remove weights for those rating is 0.
    '''
    
    user_not_rated = Original_RatingMat[Original_RatingMat[movieId] == 0].index
    
    #calculating weights.
    Total_weight = np.sum(curr_user_similarity.loc[~curr_user_similarity.index.isin(user_not_rated.tolist())]['similarity'])
    
    #appending in weighted_avg list to get final list of weighted avg movie.
    Weighed_avg.append(np.dot(RatingMatTranspose.loc[movieId],curr_user_similarity.similarity) / (Total_weight))

In [108]:
#converting Weighted list in dataframe for better operation works..

Weighed_avg = pd.DataFrame(Weighed_avg,columns=['weighted_avg'])
Weighed_avg.index = RatingMatTranspose.index

In [107]:
#Weighed_avg = Weighed_avg.loc[~Weighed_avg.index.isin(curr_user_rated_movie.movieId.tolist())]

#sort top 10 recommendation movie based on weighted avg here you can append movie count as well if you want.
Weighed_avg.sort_values(by='weighted_avg',ascending=False).head(10)
top_recommendation = Weighed_avg.sort_values(by='weighted_avg',ascending=False)

In [109]:
#validate this manually by seeing past history of current user and recommended movie list..
# as we see below current user mostly love to watch Mystrey/Thriller abd some 

#first merge MovieId in movie name and genre so we can classify based on genre and name..
Rating_Movie = Ratings.merge(Movies, left_on='movieId',right_on='movieId')
Watched_Genre = pd.DataFrame(Rating_Movie[Rating_Movie.userId==359].groupby('genres').size(),columns=['Count'])
Watched_Genre.sort_values(by='Count',ascending=False).head(10)

,Count
genres,
Drama,41
Comedy,34
Comedy|Drama|Romance,24
Comedy|Drama,24
Comedy|Romance,20
Drama|Romance,19
Drama|Thriller,19
Crime|Drama,17
Action|Adventure|Sci-Fi,16


In [110]:
#our recommendation is matching with user past history ...
top_recommendation['movieId'] = top_recommendation.index
Movies_recommended = Movies.merge(top_recommendation, left_on='movieId',right_on='movieId')
Watched_recommended_Genre = pd.DataFrame(Movies_recommended.groupby('genres').size(),columns=['Count'])
Watched_recommended_Genre.sort_values(by='Count',ascending=False).head(10)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: 'movieId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Count
genres,
Comedy,163
Drama,160
Comedy|Drama,78
Drama|Romance,74
Comedy|Romance,74
Comedy|Drama|Romance,59
Drama|Thriller,36
Comedy|Crime,25
Crime|Drama,23


# ITEM-ITEM wise collaborative filtering

In [20]:
#item wise similarity with cosine...
item_similarity = cosine_similarity(RatingMat.T)
item_sim_df = pd.DataFrame(item_similarity,index=RatingMat.columns,columns=RatingMat.columns)
item_sim_df.head(5)

#item wise similarity with pearson coorelation...
# item_similarity = 1-pairwise_distances(RatingMat.T, metric="correlation")
# item_sim_df = pd.DataFrame(item_similarity,index=RatingMat.columns,columns=RatingMat.columns)
# item_sim_df.head(5)


movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.441325,-0.156572,-0.576545,-0.158333,0.234877,-0.078188,-0.533462,0.317635,0.043216,...,-0.083434,-0.170478,-0.052600,-0.058843,-0.044648,-0.112960,-0.120081,-0.077257,-0.158921,-0.057993
2,0.441325,1.000000,-0.059794,-0.407200,-0.077746,0.156024,-0.118146,-0.298565,0.297550,0.028880,...,-0.072867,-0.128866,-0.043666,-0.086136,-0.108296,-0.125988,-0.096542,-0.081546,-0.175847,-0.093416
3,-0.156572,-0.059794,1.000000,0.151666,0.276112,-0.067516,0.109045,0.113931,-0.006632,0.161142,...,-0.005964,0.018448,0.016996,-0.022170,-0.047970,0.018375,0.031933,0.029328,0.014353,0.005830
4,-0.576545,-0.407200,0.151666,1.000000,0.195347,-0.368252,0.139833,0.585199,-0.389810,-0.039529,...,0.145793,0.210864,0.053049,0.070420,0.066963,0.116990,0.159512,0.089704,0.175868,0.102365
5,-0.158333,-0.077746,0.276112,0.195347,1.000000,-0.129055,0.218587,0.187742,-0.136468,0.208323,...,-0.029165,-0.007628,-0.044789,-0.076524,-0.050007,-0.071626,-0.078126,-0.067456,-0.049240,-0.083133


In [21]:
'''this is just example how we pick similarity for any Id later we will take similarity for all movie 1 by 1 and 
then check with specific user.. 
'''
curr_movie_similarity = pd.DataFrame(item_sim_df.loc[5])
curr_movie_similarity.rename(columns={5:'similarity'},inplace=True)
curr_movie_similarity.head()

,similarity
movieId,
1,-0.158333
2,-0.077746
3,0.276112
4,0.195347
5,1.000000


In [22]:
'''
In this we have to first calculate take similarity of movie ex similarity of movie 5 w.r.t to all and 
then multiply with user1 all movie rating rating then divide by all sum of similarity so we get weighted avg for 
movie 5. similarly for all we have to calculate. 
'''

#formulas :- sum(w[1]*movie1+w[2]*movie2.....w[n]*movieN) / w[1]+w[2]....w[n] 
Weighed_movie_avg = []
 
for movieId in item_sim_df.index:
    
    #extract similarity of particular movieId with respect to others.
    curr_movie_similarity = pd.DataFrame(item_sim_df.loc[movieId])
    
    #calculate sum of all similarity
    Total_movie_weight = np.sum(item_sim_df.loc[movieId])
    
    #calculate dot product with user 1 to similarity of particular movieId and divide by total weight.
    Weighed_movie_avg.append(np.dot(RatingMat.loc[359],curr_movie_similarity) \
                             /(np.abs(Total_movie_weight)))
    
#above loop works for all movie 1 by 1 and gives weighted avg value for all movieId

In [23]:
#convert weighted avg of all movie into frame and set column and index name..
Weighed_movie_avg = pd.DataFrame(Weighed_movie_avg)
Weighed_movie_avg.index = RatingMat.columns
Weighed_movie_avg.columns = ['weighted_avg']
Weighed_movie_avg.head()

,weighted_avg
movieId,
1,0.446238
2,0.450163
3,-0.418007
4,-0.477799
5,-0.468216


In [34]:
#sort top 10 best weighted avg score movie fot userk..
Weighed_movie_avg.sort_values(by='weighted_avg',ascending=False).head(10)
top_10_recommendation_itemwise = Weighed_movie_avg.sort_values(by='weighted_avg',ascending=False).head(10)

In [35]:
#validate this manually by seeing past history of current user and recommended movie list..
# as we see below current user mostly love to watch Mystrey/Thriller abd some 

#first merge MovieId in movie name and genre so we can classify based on genre and name..
Rating_Movie = Ratings.merge(Movies, left_on='movieId',right_on='movieId')
Rating_Movie[Rating_Movie.userId==359].head(10)

,userId,movieId,rating,timestamp,rating_count,title,genres
165,359,1,5.0,1067183252,496,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
627,359,32,3.0,1087076704,424,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
1067,359,47,4.0,1086971714,452,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
1520,359,50,4.5,1067185059,454,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
1978,359,110,3.5,1067182399,477,Braveheart (1995),Action|Drama|War
2433,359,150,4.0,1087076743,394,Apollo 13 (1995),Adventure|Drama|IMAX
2908,359,165,3.5,1082062223,274,Die Hard: With a Vengeance (1995),Action|Crime|Thriller
3273,359,260,5.0,1067183737,535,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
3827,359,296,5.0,1067185673,613,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4417,359,318,5.0,1067184946,564,"Shawshank Redemption, The (1994)",Crime|Drama


In [36]:
#our recommendation is matching with user past history ... 
Rating_Movie.loc[top_10_recommendation_itemwise.index]

,userId,movieId,rating,timestamp,rating_count,title,genres
movieId,,,,,,,
1918,97384,110,5.0,1225338354,477,Braveheart (1995),Action|Drama|War
431,53578,1,4.5,1202263442,496,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4128,90453,296,5.0,1394023441,613,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4489,83644,318,4.0,1139669618,564,"Shawshank Redemption, The (1994)",Crime|Drama
5574,17630,364,3.5,1146572322,401,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX
3555,23433,260,4.0,1310055458,535,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi
173,121612,1,3.5,1172375411,496,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4327,105059,318,4.0,1178059757,564,"Shawshank Redemption, The (1994)",Crime|Drama
1587,59424,50,5.0,1165546804,454,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
